# Finder Camera 

This notebook is about exploring the ASICamera2 API for use with the ASI120MC-S as a finder camera for remote control and exploration of the night sky.

## Setup

This project contains the ASI_linux_mac_SDK. If you need a different or updated driver please go [here](https://download.astronomy-imaging-camera.com/for-developer/). Read the [README.TXT](ASI_linux_mac_SDK_V1.20.3/lib/README.txt) for how to test that driver is installed correctly.

[Pyasi](https://github.com/j0r1/pyasi) is a [Cython](https://cython.org/) wrapper around some ASI SDK. This should allow direct calls from python.

```
sudo pip3 install cython numpy scipy opencv-python fastapi uvicorn
```

## Exploring Cython

Documentation for Cython can be found [here](https://cython.readthedocs.io/en/latest/).

Full documentation on compiling with a jupyter notebook can be found [here](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiling-notebook)


In [1]:
%load_ext Cython

In [2]:
%%cython

a: cython.int = 0
for i in range(10):
    a += i
print(a)

45


In [3]:
%%cython --annotate

cdef int a = 0
for i in range(10):
    a += i
print(a)

45


You need to place the libASICamera2 library files in the correct place so that asi.cpython-38-x86_64-linux-gnu.so can find it

```
sudo cp ASI_linux_mac_SDK_V1.20.3/lib/x64/* /lib/x86_64-linux-gnu
```



You need to install the cython asi module to your python.

```
sudo python3 setup.py install
```

If there are issues you can see the dependency locations in play:

```
ldd /home/mark/src/telescope-finder-camera/.env/lib/python3.8/site-packages/asi.cpython-38-x86_64-linux-gnu.so
```

In [4]:
import numpy as np
import scipy.misc
import asi

# List the connected camera's
cameras = asi.getConnectedCameras()
print("Connected camera's:")

for camera_number in cameras:
    print(cameras[camera_number]['Name'])
    
print(cameras)

Connected camera's:
b'ZWO ASI120MC-S(Camera)'
b'ZWO ASI482MC'
{0: {'Name': b'ZWO ASI120MC-S(Camera)', 'MaxHeight': 960, 'MaxWidth': 1280, 'IsColorCam': True, 'PixelSize': 3.75, 'MechanicalShutter': 0, 'ST4Port': 1, 'IsCoolerCam': 0, 'IsUSB3Host': 1, 'IsUSB3Camera': 1, 'ElecPerADU': 3.5199999809265137, 'BayerPattern': 'GR', 'SupportedBins': [1, 2], 'SupportedVideoFormat': ['ASI_IMG_RAW8', 'ASI_IMG_RGB24', 'ASI_IMG_Y8', 'ASI_IMG_RAW16']}, 1: {'Name': b'ZWO ASI482MC', 'MaxHeight': 1080, 'MaxWidth': 1920, 'IsColorCam': True, 'PixelSize': 5.8, 'MechanicalShutter': 0, 'ST4Port': 1, 'IsCoolerCam': 0, 'IsUSB3Host': 1, 'IsUSB3Camera': 1, 'ElecPerADU': 11.300000190734863, 'BayerPattern': 'RG', 'SupportedBins': [1, 2], 'SupportedVideoFormat': ['ASI_IMG_RAW8', 'ASI_IMG_RGB24', 'ASI_IMG_Y8', 'ASI_IMG_RAW16']}}


### Playing with exposure and gain settings based on use (Finder Camera).
Natural daylight: exposure around 300,000 us. gain is 0

Night observing (decent image feedback): exposure around 2,000,000 us. gain around 80

Night observing (long exposure): exposure around 20,000,000 us. gain around 50

In [5]:
import asi
import cv2
from typing import Optional

class VideoCamera(object):
    _video = None
    _exposure = 3000 #2000000
    _offset_cross = (-20,-18)
    _gain = 0 #82
    last_image: Optional[bytes] = None

    def __init__(self):
        self._video = asi.Camera(0)
        self._video.setCaptureFrameFormat(1280, 960, 1, "RGB24")
        self._video.setControlValueManual("ASI_GAIN", self._gain)
        self._video.setControlValueManual("ASI_WB_R", 55)
        self._video.setControlValueManual("ASI_WB_B", 77)

    def __del__(self):
        del self._video

    def get_frame(self) -> bytes:
        self.last_image, bin, success = self._video.grab(self._exposure)
        b,g,r = cv2.split(self.last_image)
        corrected_image = cv2.merge ( (r, g, b) )
        #print(self.last_image.shape)
        png = corrected_image[80-self._offset_cross[0]:880-self._offset_cross[0],240-self._offset_cross[1]:1040-self._offset_cross[1]]
        #print(png.shape)
        #ret, png = cv2.imencode('.png', self.last_image)

        cross = cv2.imread('overlays/alignment-cross-hair.png')
        #print(cross.shape)
        #overlay = cv2.imencode('.png',cv2.imread('cross-hair.png'))
        added_image = cv2.addWeighted(png,1,cross,1,0,dtype = cv2.CV_32F)
        
        ret, jpeg = cv2.imencode('.jpg', added_image)
        return jpeg.tobytes()



In [6]:
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import threading

app = FastAPI()

camera: VideoCamera = None

def run():
    uvicorn.run(app, host="0.0.0.0")
      
def start_api():
    _api_thread = threading.Thread(target=run)
    _api_thread.start()
    
def create_stream(): 
    global camera
    camera = VideoCamera()    
    while True:
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + bytearray(camera.get_frame()) + b'\r\n\r\n')
      
@app.get("/")
def get_video_stream():
    return StreamingResponse(create_stream(), media_type='multipart/x-mixed-replace; boundary=frame')

start_api()

In [12]:
%%HTML 
<img src="http://192.168.0.30:8000/" width=640/>

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 404, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 269, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-packa

In [9]:
if camera != None:
    camera._offset_cross = (-20,-18)